Variant #2

t = 100, ∆t = 10,
n1 = 150, n2 = 100, n3 = 50, n4 = 200, n5 = 100,
n6 = 100, n7 = 80, n8 = 20, n9 = 90, n10 = 10

N = n1 + n2 + . . . + n10

Failure-free operation probability P(t)

Вероятность безотказной работы P(t)

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# n-list
clients_dos_by_time_ivals = [500, 100, 50, 20, 10, 1, 8, 2, 9, 0]

# N
clients_total = np.sum(clients_dos_by_time_ivals)
display(f"Общее число абонентов: {clients_total}")

# F(t)
failure_probs = np.cumsum(clients_dos_by_time_ivals) / clients_total

# P(t) = 1 - F(t)
failure_free_probs = 1 - failure_probs

# Display dataframe
distr_table_for_P = pd.DataFrame(
    {
        "Временный интервалы (t)": np.arange(0, len(clients_dos_by_time_ivals) + 1),
        "Кол-во потерь (n)": np.insert(arr=clients_dos_by_time_ivals, obj=0, values=0),
        "P(t)": np.insert(arr=failure_free_probs, obj=0, values=1)
    }
)
display(distr_table_for_P.style.hide(axis="index"))

'Общее число абонентов: 700'

Временный интервалы (t),Кол-во потерь (n),P(t)
0,0,1.000000
1,500,0.285714
2,100,0.142857
3,50,0.071429
4,20,0.042857
5,10,0.028571
6,1,0.027143
7,8,0.015714
8,2,0.012857
9,9,0.000000


Mean Time To Failure, MTTF

Средняя наработка до отказа (среднее время безотказной работы)

In [36]:
time_ival = 10
clients_total = np.sum(clients_dos_by_time_ivals)

# Assemble an array of average work-time ivals
ivals_middle_Values_mask = (np.arange(len(clients_dos_by_time_ivals)) + 0.5)
middle_ivals = ivals_middle_Values_mask * time_ival

# Get total operational time
average_operational_time_by_ivals = clients_dos_by_time_ivals * middle_ivals
total_operational_time = np.sum(average_operational_time_by_ivals)

# Calculate MTTF
mttf = total_operational_time / clients_total

print(f"Среднее время до отказа: {mttf} минут")


Среднее время до отказа: 11.271428571428572 минут
